In [1]:
import os
import time
import subprocess
from concurrent.futures import ThreadPoolExecutor
import cv2
from datetime import datetime


bitrate_map = {
    "360p_SD": "500k",
    "720p_HD": "1000k",
    "1080p_FHD": "3000k",
    "2K_UHD": "6000k",
    "4K_FULL": "10000k"
}

resolution_settings = [
    ("360p_SD", 640, 360, 500000),
    ("720p_HD", 1280, 720, 1000000),
    ("1080p_FHD", 1920, 1080, 3000000),
    ("2K_UHD", 2560, 1440, 6000000),
    ("4K_FULL", 3840, 2160, 10000000)
]

def process_resolution(resolution_key, width, height, list_file, output_dir):
    segment_pattern = os.path.join(output_dir, f"{resolution_key}_%03d.ts")
    output_m3u8 = os.path.join(output_dir, f"{resolution_key}.m3u8")
    bitrate = bitrate_map[resolution_key]

    command = [
        'ffmpeg', '-y', '-f', 'concat', '-safe', '0', '-i', list_file,
        '-vf', f"scale={width}:{height}", '-r', '8',
        '-c:v', 'libx264', '-preset', 'veryfast', '-pix_fmt', 'yuv420p',
        '-profile:v', 'main', '-g', '40', '-keyint_min', '40', '-sc_threshold', '0',
        '-b:v', bitrate, '-maxrate', bitrate, '-bufsize', '2000k',
        '-an',
        '-f', 'hls', '-hls_time', '5', '-hls_list_size', '0',
        '-hls_segment_filename', segment_pattern,
        output_m3u8
    ]

    try:
        subprocess.run(command, check=True, capture_output=True, text=True)
        print(f"[{resolution_key}]  M3U8 created: {output_m3u8}")
    except subprocess.CalledProcessError as e:
        print(f"[{resolution_key}]  FFmpeg error: {e.stderr}")

def create_master_playlist(output_dir, resolutions):
    master_path = os.path.join(output_dir, "master.m3u8")
    with open(master_path, "w") as f:
        f.write("#EXTM3U\n")
        f.write("#EXT-X-VERSION:3\n")
        f.write("#EXT-X-INDEPENDENT-SEGMENTS\n")
        for res_key, width, height, bandwidth in resolutions:
            m3u8_file = f"{res_key}.m3u8"
            f.write(f"#EXT-X-STREAM-INF:BANDWIDTH={bandwidth},RESOLUTION={width}x{height}\n")
            f.write(f"{m3u8_file}\n")
    print(f" Master playlist created: {master_path}")


def add_timestamp_text_to_images(image_folder):
    image_files = sorted([f for f in os.listdir(image_folder) if f.lower().endswith('.jpeg')])
    
    for img_file in image_files:
        img_path = os.path.join(image_folder, img_file)
        img = cv2.imread(img_path)

        try:
            timestamp_str = os.path.splitext(img_file)[0]
            timestamp = datetime.strptime(timestamp_str, "%Y-%m-%d-%H-%M-%S")
            overlay_text = timestamp.strftime("%Y-%m-%d %H:%M:%S")
        except Exception as e:
            print(f" Skipping file {img_file}: {e}")
            continue


        font = cv2.FONT_HERSHEY_SIMPLEX
        position = (30, img.shape[0] - 30)
        font_scale = 1
        color = (255, 255, 255) 
        thickness = 2
        outline_color = (0, 0, 0)

        cv2.putText(img, overlay_text, position, font, font_scale, outline_color, thickness + 2, cv2.LINE_AA)
        
        cv2.putText(img, overlay_text, position, font, font_scale, color, thickness, cv2.LINE_AA)

        cv2.imwrite(img_path, img)

def images_to_m3u8(image_folder, output_dir, fps=8, threads=8):
    start_time = time.time()

    if not os.path.exists(image_folder):
        print(f" Folder not found: {image_folder}")
        return

    os.makedirs(output_dir, exist_ok=True)

    image_files = sorted([f for f in os.listdir(image_folder) if f.lower().endswith('.jpeg')])
    if not image_files:
        print(" No JPEG images found.")
        return

    print(" Adding timestamps to images...")
    add_timestamp_text_to_images(image_folder)  

    list_file = os.path.join(output_dir, "file_list.txt")
    with open(list_file, "w") as f:
        for img in image_files:
            img_path = os.path.join(image_folder, img).replace('\\', '/')
            f.write(f"file '{img_path}'\n")

    with ThreadPoolExecutor(max_workers=threads) as executor:
        futures = []
        for res_key, w, h, _ in resolution_settings:
            futures.append(executor.submit(
                process_resolution, res_key, w, h,
                list_file, output_dir
            ))
        for future in futures:
            future.result()

    create_master_playlist(output_dir, resolution_settings)
    os.remove(list_file)
    print(f" All done in {time.time() - start_time:.2f}s")

if __name__ == "__main__":
    image_folder = "C:/Users/jl644/Desktop/combined_ims"
    output_dir = "C:/Users/jl644/Desktop/output"
    images_to_m3u8(image_folder, output_dir, threads=8)


 Adding timestamps to images...
[720p_HD]  M3U8 created: C:/Users/jl644/Desktop/output\720p_HD.m3u8
[2K_UHD]  M3U8 created: C:/Users/jl644/Desktop/output\2K_UHD.m3u8
[360p_SD]  M3U8 created: C:/Users/jl644/Desktop/output\360p_SD.m3u8
[4K_FULL]  M3U8 created: C:/Users/jl644/Desktop/output\4K_FULL.m3u8
[1080p_FHD]  M3U8 created: C:/Users/jl644/Desktop/output\1080p_FHD.m3u8
 Master playlist created: C:/Users/jl644/Desktop/output\master.m3u8
 All done in 130.48s


In [2]:
pip install -U yt-dlp


Note: you may need to restart the kernel to use updated packages.


In [3]:
import os
import subprocess

resolutions = ["360p_SD", "720p_HD", "1080p_FHD", "2K_UHD", "4K_FULL"]
base_path = "C:/Users/jl644/Desktop/output"
ts_base_path = f"file:///{base_path.replace(os.sep, '/')}"  

for res in resolutions:
    m3u8_input_path = os.path.join(base_path, f"{res}.m3u8")
    m3u8_output_path = os.path.join(base_path, f"{res}_absolute.m3u8")

    with open(m3u8_input_path, 'r', encoding='utf-8') as f:
        lines = f.readlines()

    new_lines = []
    for line in lines:
        if line.strip().endswith(".ts"):
            ts_file = line.strip()
            full_url = f"{ts_base_path}/{ts_file}"
            new_lines.append(full_url + "\n")
        else:
            new_lines.append(line)

    with open(m3u8_output_path, 'w', encoding='utf-8') as f:
        f.writelines(new_lines)

    print(f" create absolute M3U8: {m3u8_output_path}")

    output_mp4_path = os.path.join(base_path, f"{res}_output.mp4")
    cmd = [
        "yt-dlp",
        "--enable-file-urls",
        f"file:///{m3u8_output_path.replace(os.sep, '/')}",
        "-o", output_mp4_path
    ]
    
    result = subprocess.run(cmd, capture_output=True, text=True)
    print(f"\n {res} output path: {output_mp4_path}")
    print("STDOUT:\n", result.stdout)
    print("STDERR:\n", result.stderr)
    print("Return Code:", result.returncode)


 create absolute M3U8: C:/Users/jl644/Desktop/output\360p_SD_absolute.m3u8

 360p_SD output path: C:/Users/jl644/Desktop/output\360p_SD_output.mp4
STDOUT:
 [generic] Extracting URL: file:///C:/Users/jl644/Desktop/output/360p_SD_absolute.m3u8
[generic] 360p_SD_absolute: Downloading webpage
[generic] 360p_SD_absolute: Downloading m3u8 information
[generic] 360p_SD_absolute: Checking m3u8 live status
[info] 360p_SD_absolute: Downloading 1 format(s): 0
[download] C:\Users\jl644\Desktop\output\360p_SD_output.mp4 has already been downloaded

[download] 100% of  503.02KiB

STDERR:
 
Return Code: 0
 create absolute M3U8: C:/Users/jl644/Desktop/output\720p_HD_absolute.m3u8

 720p_HD output path: C:/Users/jl644/Desktop/output\720p_HD_output.mp4
STDOUT:
 [generic] Extracting URL: file:///C:/Users/jl644/Desktop/output/720p_HD_absolute.m3u8
[generic] 720p_HD_absolute: Downloading webpage
[generic] 720p_HD_absolute: Downloading m3u8 information
[generic] 720p_HD_absolute: Checking m3u8 live status
[

In [4]:
html_content = """<!DOCTYPE html>
<html>
<head>
  <meta charset="utf-8">
  <title>HLS Manual Resolution Switch</title>
</head>
<body>
  <h2>HLS Video Player (Manual Resolution Switch)</h2>
  <video id="video" width="800" height="450" controls></video>
  <br>
  <label for="qualitySelect">Switch Resolution:</label>
  <select id="qualitySelect">
    <option value="-1">Auto</option>
  </select>

  <script src="https://cdn.jsdelivr.net/npm/hls.js@latest"></script>
  <script>
    const video = document.getElementById('video');
    const select = document.getElementById('qualitySelect');
    const videoSrc = "http://127.0.0.1:8080/master.m3u8";

    if (Hls.isSupported()) {
      const hls = new Hls();
      hls.loadSource(videoSrc);
      hls.attachMedia(video);

      hls.on(Hls.Events.MANIFEST_PARSED, function (_, data) {
        const levels = data.levels;

        // Add options for each resolution
        levels.forEach((level, index) => {
          const option = document.createElement('option');
          option.value = index;
          option.text = `${level.height}p (${Math.round(level.bitrate / 1000)} kbps)`;
          select.appendChild(option);
        });

        select.addEventListener('change', () => {
          const selectedIndex = parseInt(select.value);
          hls.currentLevel = selectedIndex;  // -1 means "auto"
        });
      });
    } else if (video.canPlayType('application/vnd.apple.mpegurl')) {
      video.src = videoSrc;
    } else {
      alert("Your browser does not support HLS.");
    }
  </script>
</body>
</html>
"""

with open("C:/Users/jl644/Desktop/output/hls_player_manual.html", "w", encoding="utf-8") as f:
    f.write(html_content)

print(" HTML player saved to: C:/Users/jl644/Desktop/output/hls_player_manual.html")


 HTML player saved to: C:/Users/jl644/Desktop/output/hls_player_manual.html
